In [260]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras import layers

In [261]:
data = pd.read_csv("train.csv")
data.pop('Id')
data = data.dropna(subset=['SalePrice'])


string_features_columns = data.select_dtypes('object').columns
numeric = data.copy()
for x in string_features_columns:
    numeric.pop(x)

categorical = data.select_dtypes('object')

In [262]:
enc = OneHotEncoder().fit(categorical)
enc.categories_
categorical_transformed = pd.DataFrame(enc.transform(categorical).toarray())

In [263]:
label = numeric.pop('SalePrice')

In [264]:
def min_max_scaling(series):
    return (series - series.min()) / (series.max() - series.min())
for col in numeric.columns:
    numeric[col] = min_max_scaling(numeric[col])

In [265]:
features = numeric.join(categorical_transformed).fillna(0)
val_f = features.tail(100)

val_l = label.tail(100)

features = features.head(-100)
label = label.head(-100)

In [266]:
x=0.0
while x>288874336.0 or x==0.0:
    model = tf.keras.Sequential([
            layers.Dense(144),
            tf.keras.layers.Dropout(0.2),

            layers.Dense(256),
            tf.keras.layers.Dropout(0.2),
            
            layers.Dense(144),
            tf.keras.layers.Dropout(0.1),
            
            layers.Dense(10),
            tf.keras.layers.Dropout(0.2),

            layers.Dense(1)
        ])
    model.compile(loss = tf.keras.losses.MeanSquaredError(),
                        optimizer = tf.optimizers.Adam())

    model.fit(x=features,y=label, epochs=768,verbose=0)
    x = model.evaluate(x=val_f,y=val_l)

4/4 [==============================] - 0s 1ms/step - loss: 277960000.0000


In [267]:

data = pd.read_csv("test.csv")
ids = data.pop('Id')


In [268]:
string_features_columns = data.select_dtypes('object').columns
numeric = data.copy()
for x in string_features_columns:
    numeric.pop(x)

categorical = data.select_dtypes('object')

enc = OneHotEncoder().fit(categorical)
enc.categories_
categorical_transformed = pd.DataFrame(enc.transform(categorical).toarray())

def min_max_scaling(series):
    return (series - series.min()) / (series.max() - series.min())
for col in numeric.columns:
    numeric[col] = min_max_scaling(numeric[col])

features = numeric.join(categorical_transformed).fillna(0)

preds = model.predict(features)

In [269]:
preds = preds.flatten()

In [270]:
submission = pd.concat([ids,pd.Series(preds,name="SalePrice")],axis=1)
submission.to_csv("./submission.csv",index=False,header=True)